In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import os
import glob

<IPython.core.display.Javascript object>

In [3]:
!ls all_cements/grouped

BidirectionalGRU_results_grouped_10.csv   lstm_results_grouped_1.csv
BidirectionalGRU_results_grouped_1.csv	  lstm_results_grouped_2.csv
BidirectionalGRU_results_grouped_2.csv	  lstm_results_grouped_3.csv
BidirectionalGRU_results_grouped_3.csv	  lstm_results_grouped_4.csv
BidirectionalGRU_results_grouped_4.csv	  lstm_results_grouped_5.csv
BidirectionalGRU_results_grouped_5.csv	  lstm_results_grouped_6.csv
BidirectionalGRU_results_grouped_6.csv	  lstm_results_grouped_7.csv
BidirectionalGRU_results_grouped_7.csv	  lstm_results_grouped_8.csv
BidirectionalGRU_results_grouped_8.csv	  lstm_results_grouped_9.csv
BidirectionalGRU_results_grouped_9.csv	  mlp_results_grouped_10.csv
BidirectionalLSTM_results_grouped_10.csv  mlp_results_grouped_1.csv
BidirectionalLSTM_results_grouped_1.csv   mlp_results_grouped_2.csv
BidirectionalLSTM_results_grouped_2.csv   mlp_results_grouped_3.csv
BidirectionalLSTM_results_grouped_3.csv   mlp_results_grouped_4.csv
BidirectionalLSTM_results_grouped_4.csv   mlp_r

<IPython.core.display.Javascript object>

### Reading The files

In [4]:
path = "/home/peressim/projects/ccs28-ml-modelling/reports/results/local_models/partner_i-oficial/all_cements/grouped/"
csv_files = glob.glob(os.path.join(path, "*.csv"))

<IPython.core.display.Javascript object>

In [5]:
results = []

for filepath in csv_files:
    df = pd.read_csv(filepath, header=[0, 1])
    results.append(df)
df = pd.concat(results, ignore_index=True)

<IPython.core.display.Javascript object>

### Renaming the Columns

In [6]:
df.columns = pd.MultiIndex.from_tuples(
    list(
        {
            col: (col[0], "") if "Unnamed" in col[1] else col for col in df.columns
        }.values()
    )
)

<IPython.core.display.Javascript object>

### Defining the Columns to work with

In [7]:
cols = [
    ("Model", ""),
    ("Features", ""),
    ("Cross Validation", ""),
    ("Cross Validation Params", ""),
    ("RMSE Test", "mean"),
    ("RMSE Test", "std"),
    ("MAE Test", "mean"),
    ("MAE Test", "std"),
    ("MAPE Test", "mean"),
    ("MAPE Test", "std"),
    ("R2 Test", "mean"),
    ("R2 Test", "std"),
]

<IPython.core.display.Javascript object>

In [8]:
cols_metrics = [
    ("RMSE Test", "mean"),
    ("RMSE Test", "std"),
    ("MAE Test", "mean"),
    ("MAE Test", "std"),
    ("MAPE Test", "mean"),
    ("MAPE Test", "std"),
    ("R2 Test", "mean"),
    ("R2 Test", "std"),
]

<IPython.core.display.Javascript object>

# Linear Regression

In [9]:
df_linreg = df[df[("Model", "")] == "Linear Regression"][cols].copy()
# df_linreg = df_linreg.drop([("Model", "")], axis=1)
df_linreg = df_linreg[df_linreg["Cross Validation"] != "Out of time Split"]

<IPython.core.display.Javascript object>

In [10]:
df_linreg[df_linreg["Features"] == "Chemical + Mineralogical"]

Model                  Features            Cross Validation  \
                                                                               
344  Linear Regression  Chemical + Mineralogical  Blocking Time Series Split   
346  Linear Regression  Chemical + Mineralogical              Repeated KFold   
347  Linear Regression  Chemical + Mineralogical           Time Series Split   

                              Cross Validation Params RMSE Test            \
                                                           mean       std   
344  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}  3.844932  0.295179   
346                     {"N_Splits": 5, "Repeats": 3}  3.564466  0.152389   
347           {"N_Splits": 5, "Repeats": 1, "Gap": 0}  4.166360  0.608344   

     MAE Test           MAPE Test             R2 Test            
         mean       std      mean       std      mean       std  
344  2.945173  0.289977  0.069038  0.006380  0.383375  0.105745  
346  2.755747  0.095292  0.065881  0.002301  0.495284  0.054956  
347  3.217591  0.453831  0.076196  0.009931  0.309890  0.174562

<IPython.core.display.Javascript object>

In [11]:
idx_min = df_linreg[df_linreg["Features"] == "Chemical + Mineralogical"][
    "RMSE Test", "mean"
].idxmin()

df_linreg.loc[[idx_min]].round(3)

Model                  Features Cross Validation  \
                                                                    
346  Linear Regression  Chemical + Mineralogical   Repeated KFold   

           Cross Validation Params RMSE Test        MAE Test        MAPE Test  \
                                        mean    std     mean    std      mean   
346  {"N_Splits": 5, "Repeats": 3}     3.564  0.152    2.756  0.095     0.066   

           R2 Test         
       std    mean    std  
346  0.002   0.495  0.055

<IPython.core.display.Javascript object>

In [12]:
exclude = r"(?!.*One-Hot)"
pattern1 = r"Chemical \+ Mineralogical \+ CS*"
pattern2 = r"Chemical \+ Mineralogical \+ Physical"
pattern = "(" + pattern1 + "|" + pattern2 + ")" + exclude

features_filter = df_linreg["Features"].str.contains(
    pattern,
    regex=True,
)

idx_min = df_linreg[features_filter].round(3)["RMSE Test", "mean"].idxmin()

df_linreg.loc[[idx_min]].round(3)

/tmp/ipykernel_2546721/383223557.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  features_filter = df_linreg["Features"].str.contains(


Model                              Features Cross Validation  \
                                                                                
698  Linear Regression  Chemical + Mineralogical + CS3 + CS7   Repeated KFold   

           Cross Validation Params RMSE Test        MAE Test        MAPE Test  \
                                        mean    std     mean    std      mean   
698  {"N_Splits": 5, "Repeats": 3}     1.065  0.094    0.803  0.027     0.019   

           R2 Test         
       std    mean    std  
698  0.001   0.955  0.009

<IPython.core.display.Javascript object>

In [13]:
idx_min = df_linreg[df_linreg["Features"] == "Chemical + Mineralogical + One-Hot"][
    "RMSE Test", "mean"
].idxmin()

df_linreg.loc[[idx_min]].round(3)

Model                            Features  \
                                                             
420  Linear Regression  Chemical + Mineralogical + One-Hot   

               Cross Validation  \
                                  
420  Blocking Time Series Split   

                              Cross Validation Params RMSE Test         \
                                                           mean    std   
420  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}       1.0  0.181   

    MAE Test        MAPE Test        R2 Test         
        mean    std      mean    std    mean    std  
420    0.771  0.111     0.018  0.002   0.958  0.013

<IPython.core.display.Javascript object>

In [14]:
pattern = r"One-Hot"

features_filter = df_linreg["Features"].str.contains(
    pattern,
    regex=True,
)

idx_min = df_linreg[features_filter].round(3)["RMSE Test", "mean"].idxmin()

df_linreg.loc[[idx_min]].round(3)

Model                                  Features  \
                                                                   
550  Linear Regression  Chemical + Mineralogical + CS7 + One-Hot   

    Cross Validation        Cross Validation Params RMSE Test        MAE Test  \
                                                         mean    std     mean   
550   Repeated KFold  {"N_Splits": 5, "Repeats": 3}     0.906  0.035    0.711   

           MAPE Test      R2 Test         
       std      mean  std    mean    std  
550  0.022     0.016  0.0   0.967  0.003

<IPython.core.display.Javascript object>

# Neural Networks

In [15]:
neural_networks = [
    "MLP",
    "LSTM",
    "GRU",
    "BidirectionalGRU",
    "BidirectionalLSTM",
    "Transformer",
]

df_nn = df[df[("Model", "")].isin(neural_networks)][cols].copy()
# df_nn = df_nn.drop([("Model", "")], axis=1)
df_nn = df_nn[df_nn["Cross Validation"] != "Out of time Split"]

<IPython.core.display.Javascript object>

In [16]:
idx_min = df_nn[df_nn["Features"] == "Chemical + Mineralogical"][
    "RMSE Test", "mean"
].idxmin()

df_nn.loc[[idx_min]].round(3)

Model                  Features Cross Validation  \
                                                            
6  Transformer  Chemical + Mineralogical   Repeated KFold   

         Cross Validation Params RMSE Test        MAE Test        MAPE Test  \
                                      mean    std     mean    std      mean   
6  {"N_Splits": 5, "Repeats": 3}     2.248  0.393    1.694  0.238      0.04   

         R2 Test         
     std    mean    std  
6  0.006   0.794  0.078

<IPython.core.display.Javascript object>

In [17]:
exclude = r"(?!.*One-Hot)"
pattern1 = r"Chemical \+ Mineralogical \+ CS*"
pattern2 = r"Chemical \+ Mineralogical \+ Physical"
pattern = "(" + pattern1 + "|" + pattern2 + ")" + exclude

# x = df_nn["Features"] != "Chemical + Mineralogical + Physical"
features_filter = df_nn["Features"].str.contains(
    pattern,
    regex=True,
)

idx_min = df_nn[features_filter].round(3)["RMSE Test", "mean"].idxmin()

df_nn.loc[[idx_min]].round(3)

/tmp/ipykernel_2546721/1917742884.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  features_filter = df_nn["Features"].str.contains(


Model                             Features Cross Validation  \
                                                                        
66  Transformer  Chemical + Mineralogical + Physical   Repeated KFold   

          Cross Validation Params RMSE Test        MAE Test        MAPE Test  \
                                       mean    std     mean    std      mean   
66  {"N_Splits": 5, "Repeats": 3}     1.073  0.104    0.828  0.061     0.019   

          R2 Test         
      std    mean    std  
66  0.001   0.954  0.009

<IPython.core.display.Javascript object>

In [18]:
idx_min = df_nn[df_nn["Features"] == "Chemical + Mineralogical + One-Hot"][
    "RMSE Test", "mean"
].idxmin()

df_nn.loc[[idx_min]].round(3)

Model                            Features Cross Validation  \
                                                                       
78  Transformer  Chemical + Mineralogical + One-Hot   Repeated KFold   

          Cross Validation Params RMSE Test        MAE Test        MAPE Test  \
                                       mean    std     mean    std      mean   
78  {"N_Splits": 5, "Repeats": 3}     1.136  0.063    0.877  0.044      0.02   

          R2 Test         
      std    mean    std  
78  0.001   0.949  0.005

<IPython.core.display.Javascript object>

In [19]:
pattern = r"One-Hot"

features_filter = df_nn["Features"].str.contains(
    pattern,
    regex=True,
)

idx_min = df_nn[features_filter].round(3)["RMSE Test", "mean"].idxmin()

df_nn.loc[[idx_min]].round(3)

Model                                  Features Cross Validation  \
                                                                              
494  Transformer  Chemical + Mineralogical + CS7 + One-Hot   Repeated KFold   

           Cross Validation Params RMSE Test        MAE Test        MAPE Test  \
                                        mean    std     mean    std      mean   
494  {"N_Splits": 5, "Repeats": 3}     1.048  0.104    0.824  0.102     0.019   

           R2 Test        
       std    mean   std  
494  0.002   0.956  0.01

<IPython.core.display.Javascript object>

# Trees

In [20]:
trees = ["Decision Tree", "Random Forest", "XGBoost"]

df_trees = df[df[("Model", "")].isin(trees)][cols].copy()
df_trees = df_trees[df_trees["Cross Validation"] != "Out of time Split"]

<IPython.core.display.Javascript object>

In [21]:
idx_min = df_trees[df_trees["Features"] == "Chemical + Mineralogical"][
    "RMSE Test", "mean"
].idxmin()

df_trees.loc[[idx_min]].round(3)

Model                  Features            Cross Validation  \
                                                                     
268  XGBoost  Chemical + Mineralogical  Blocking Time Series Split   

                              Cross Validation Params RMSE Test         \
                                                           mean    std   
268  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}     3.271  0.427   

    MAE Test        MAPE Test        R2 Test         
        mean    std      mean    std    mean    std  
268    2.477  0.304     0.059  0.008   0.546  0.133

<IPython.core.display.Javascript object>

In [22]:
exclude = r"(?!.*One-Hot)"
pattern1 = r"Chemical \+ Mineralogical \+ CS*"
pattern2 = r"Chemical \+ Mineralogical \+ Physical"
pattern = "(" + pattern1 + "|" + pattern2 + ")" + exclude

# x = df_nn["Features"] != "Chemical + Mineralogical + Physical"
features_filter = df_trees["Features"].str.contains(
    pattern,
    regex=True,
)

idx_min = df_trees[features_filter].round(3)["RMSE Test", "mean"].idxmin()

df_trees.loc[[idx_min]].round(3)

/tmp/ipykernel_2546721/2477923481.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  features_filter = df_trees["Features"].str.contains(


Model                             Features  \
                                                          
716  Random Forest  Chemical + Mineralogical + Physical   

               Cross Validation  \
                                  
716  Blocking Time Series Split   

                              Cross Validation Params RMSE Test         \
                                                           mean    std   
716  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}     1.011  0.269   

    MAE Test        MAPE Test        R2 Test         
        mean    std      mean    std    mean    std  
716    0.774  0.196     0.018  0.005   0.956  0.022

<IPython.core.display.Javascript object>

In [23]:
idx_min = df_trees[df_trees["Features"] == "Chemical + Mineralogical + One-Hot"][
    "RMSE Test", "mean"
].idxmin()

df_trees.loc[[idx_min]].round(3)

Model                            Features  \
                                                         
404  Random Forest  Chemical + Mineralogical + One-Hot   

               Cross Validation  \
                                  
404  Blocking Time Series Split   

                              Cross Validation Params RMSE Test         \
                                                           mean    std   
404  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}     1.042  0.349   

    MAE Test        MAPE Test        R2 Test         
        mean    std      mean    std    mean    std  
404    0.766  0.246     0.018  0.006   0.949  0.038

<IPython.core.display.Javascript object>

In [24]:
pattern = r"One-Hot"

features_filter = df_trees["Features"].str.contains(
    pattern,
    regex=True,
)

idx_min = df_trees[features_filter].round(3)["RMSE Test", "mean"].idxmin()

df_trees.loc[[idx_min]].round(3)

Model                                  Features  \
                                                               
424  Random Forest  Chemical + Mineralogical + CS3 + One-Hot   

               Cross Validation  \
                                  
424  Blocking Time Series Split   

                              Cross Validation Params RMSE Test         \
                                                           mean    std   
424  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}     0.949  0.217   

    MAE Test        MAPE Test        R2 Test         
        mean    std      mean    std    mean    std  
424    0.709  0.138     0.016  0.003   0.962  0.016

<IPython.core.display.Javascript object>